In [1]:
import torch, torchvision
import sys # Python system library needed to load custom functions
import math # module with access to mathematical functions
import os # for changing the directory

import numpy as np  # for performing calculations on numerical arrays
import pandas as pd  # home of the DataFrame construct, _the_ most important object for Data Science
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt  # allows creation of insightful plots

sys.path.append('../../audio_preprocessing')
sys.path.append('../../src')
sys.path.append('../../model_training_utils')


from normalised_image_dataset import NormalisedImageDataSet
from gdsc_utils import PROJECT_DIR
import model_training
import model_eval

os.chdir(PROJECT_DIR) # changing our directory to root

In [2]:
df = pd.read_csv('data/normalised_image_train_val_with_upsample.csv')
df.head()

,Unnamed: 0,file_path,label
0,0,data/image_train_val_with_upsample/0.pt,56
1,1,data/image_train_val_with_upsample/1.pt,56
2,2,data/image_train_val_with_upsample/2.pt,56
3,3,data/image_train_val_with_upsample/3.pt,56
4,4,data/image_train_val_with_upsample/4.pt,56


In [3]:
train_df_list = []
val_df_list = []

for i in range(66):
    my_df = df[df["label"] == i]
    current_train_df, current_val_df = train_test_split(my_df, test_size=0.2)
    train_df_list.append(current_train_df)
    val_df_list.append(current_val_df)

df_train = pd.concat(train_df_list, ignore_index=True)
df_val = pd.concat(val_df_list, ignore_index=True)

In [4]:
df_train.to_csv("model_definition_and_training/EfficientNet/efficient_net_train.csv", index=False)
df_val.to_csv("model_definition_and_training/EfficientNet/efficient_net_val.csv", index=False)

In [5]:
train_dataset = NormalisedImageDataSet(
    df=df_train, 
    shuffle=True
)

val_dataset = NormalisedImageDataSet(
    df=df_val, 
    shuffle=False
)

train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=28)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=28)

In [6]:
device = model_training.get_device()

In [9]:
from torchvision.models import efficientnet_b3
import torch.optim as optim
import torch.nn as nn

#resnet_model = resnet34(weights=ResNet34_Weights.DEFAULT)
efficient_net_model = efficientnet_b3(num_classes=66, weights=None)
efficient_net_model.features[0][0] = nn.Conv2d(1, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
efficient_net_model = efficient_net_model.to(device)

In [10]:
print(efficient_net_model)

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(1, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [11]:
optimizer = optim.Adam(efficient_net_model.parameters(), amsgrad=True)
loss = nn.CrossEntropyLoss()

In [19]:
model_training.training(
    model=efficient_net_model, 
    optimizer=optimizer, 
    loss_fn=loss, 
    train_dataloader=train_dataloader, 
    val_dataloader=val_dataloader, 
    model_path="models/EfficientNet_train_val_new", 
    start_epoch=10,
    early_stop_thresh=10,
)

KeyboardInterrupt: 

In [20]:
torch.save(efficient_net_model, 'models/EfficientNet_train_val_new/efficient_model_epoch_10.pth')